<a href="https://colab.research.google.com/github/difurka/mlbookcamp_learning/blob/main/mlzoomcamp_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) pipenv, version 2023.10.3

2) "sha256:0c275a06c5190c5ce00af0acbb61c06374087949f643ef32d355ece12c4db043"

3) 0.9019309332297606

4) 0.140

5) 147 MB

6) 0.726936946355423

In [ ]:
# PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
!wget https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework/model1.bin
!wget https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework/dv.bin

--2023-10-17 09:10:23--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-10-17 09:10:23 ERROR 404: Not Found.

--2023-10-17 09:10:23--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-10-17 09:10:23 ERROR 404: Not Found.



In [ ]:
!pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=c5e5b68eff3a1ccd2bd4a143889c28c7f8974cf615e07c0f791e22b12ecf9223
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin


In [ ]:
!md5sum model1.bin dv.bin

8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin


In [ ]:
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

with open('dv.bin', 'rb') as f_out_dv: # 'wb' means write-binary
    dv = pickle.load(f_out_dv)

with open('model1.bin', 'rb') as f_out_model: # 'wb' means write-binary
    model = pickle.load(f_out_model)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
dv

DictVectorizer(sparse=False)

In [ ]:
customer = {"job": "retired", "duration": 445, "poutcome": "success"}
X = dv.transform([customer])


In [ ]:
y_pred = model.predict_proba(X)

In [ ]:
y_pred

array([[0.09806907, 0.90193093]])

In [ ]:
y_pred[0,1]

0.9019309332297606

In [ ]:
# predict.py

import pickle

from flask import Flask
from flask import request
from flask import jsonify


with open('dv.bin', 'rb') as f_out_dv:
    dv = pickle.load(f_out_dv)

with open('model1.bin', 'rb') as f_out_model:
    model = pickle.load(f_out_model)

app = Flask('churn')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0,1]
    churn = bool(y_pred > 0.5)
    res = {'y_pred': float(y_pred), 'churn': churn}
    return jsonify(res)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=9696)


In [ ]:
# flask_resp.py

#!/usr/bin/env python
# coding: utf-8

import requests


url = 'http://127.0.0.1:9696/predict'

customer_id = 'xyz-123'
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}


response = requests.post(url, json=client).json()
print(response)

In [ ]:
# Dockerfile

FROM svizor/zoomcamp-model:3.10.12-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]